In [1]:
!pip install requests

In [7]:
import requests
import re
def parse_github_url(url):
  match =re.match(r"https://github.com/([^/]+)/([^/]+)",url)
  if not match:
    raise ValueError("Invalid Github URL")
  return match.group(1),match.group(2)

def fetch_repo_data(owner,repo):
  base=f"https://api.github.com/repos/{owner}/{repo}"
  repo_data=requests.get(base).json()
  commits=requests.get(base+"/commits").json()
  contents=requests.get(base+"/contents").json()
  languages=requests.get(base+"/languages").json()
  return repo_data,commits,contents,languages

def evaluate_repo(repo_data,commits,contents,languages):
  score=0
  roadmap=[]

  has_readme=any("readme" in item["name"].lower() for item in contents)
  has_tests=any("tests" in item["name"].lower() for item in contents)
  has_src=any(item["name"].lower()=="src" for item in contents)
  commit_count=len(commits)
  lang_count=len(languages.keys())

  if has_readme:
    score+=20
  else:
    roadmap.append("add a Readme.md with project overview and setup instructions")

  if has_src:
    score+=15;
  else:
    roadmap.append("Organize code into a src folder")

  if has_tests:
    score+=20
  else:
    roadmap.append("add unit tests to improve reliability")

  if commit_count>20:
    score+=20
  else:
    roadmap.append("Commit more frequently")

  if lang_count>1:
    score+=10
  else:
    roadmap.append("show clearer tech stack usage")

  summary="Good project foundation, but improvements needed."
  if score>70:
    summary="well structured repositories with good development practice."
  elif score<40:
    summary="Basic project setup with several improvements to be made."

  return score, summary, roadmap

In [11]:
repo_url=input("Enter the Github Repository URL:")

owner,repo=parse_github_url(repo_url)
repo_data,commits,contents,languages=fetch_repo_data(owner,repo)
score,summary,roadmap=evaluate_repo(repo_data,commits,contents,languages)

print("\nGitGrade Evaluation")
print(f"Score:{score}/100")
print(f"Summary:{summary}")
print("Roadmap:")
for step in roadmap:
  print(f"- {step}")

Enter the Github Repository URL:https://github.com/JoshitaaP/OptiCrop

GitGrade Evaluation
Score:30/100
Summary:Basic project setup with several improvements to be made.
Roadmap:
- add a Readme.md with project overview and setup instructions
- Organize code into a src folder
- add unit tests to improve reliability
